# Load teacher model, news data and conll data

In [1]:
path = ''
import flair
print(flair.__version__)

from flair.models import SequenceTagger
teacher = SequenceTagger.load("flair/ner-german-large")

import json
import numpy as np
sentences_news = json.load(open(path+'data/news_12.5k.json', 'r', encoding='utf-8'))
sentences_news = [sent for sent in sentences_news if sent != ''] # filter out empty sentences

print(np.histogram([len(s) for s in sentences_news]))

from flair.datasets import CONLL_03_GERMAN
corpus = CONLL_03_GERMAN(base_path = path+'data/' ,encoding= 'latin-1' )
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')

print('len(news data)',len(sentences_news))
print('len(conll corpus train)',len(corpus.train))
print('len(conll corpus test',len(corpus.test))
print('len(conll corpus dev)',len(corpus.dev))


0.8.0.post1
2021-08-20 10:55:12,860 loading file /Users/Wu/.flair/models/ner-german-large/6b8de9edd73722050be2547acf64c037b2df833c6e8f0e88934de08385e26c1e.4b0797effcc6ebb1889d5d29784b97f0a099c1569b319d87d7c387e44e2bba48
(array([1122, 3387, 3433, 2387, 1239,  571,  290,  149,   77,   49]), array([  1. ,  45.8,  90.6, 135.4, 180.2, 225. , 269.8, 314.6, 359.4,
       404.2, 449. ]))
2021-08-20 10:55:31,146 Reading data from data/conll_03_german
2021-08-20 10:55:31,148 Train: data/conll_03_german/deu.train
2021-08-20 10:55:31,148 Dev: data/conll_03_german/deu.dev
2021-08-20 10:55:31,149 Test: data/conll_03_german/deu.testb
len(news data) 12704
len(conll corpus train) 12705
len(conll corpus test 3160
len(conll corpus dev) 93


# prepare logits results and get output --- conll data

In [43]:
from flair.data import Sentence
from tqdm import tqdm
import torch

sentences_conll = []
for sent in corpus.train:
    sentences_conll.append(sent.to_original_text())

logits = []
for sent in tqdm(sentences_conll):
    sent = Sentence(sent) 
    feature = teacher.forward([sent]).detach() #NOTE: forward function takes batch as input, here seen as batch_size=1, add extra list dim.
    logits.append(feature.squeeze(0)) #NOTE: squeeze the additional 1 dim caused by batch_size 1, but not the additional 1 dim caused by len(sentence) = 1

100%|██████████| 12705/12705 [1:13:26<00:00,  2.88it/s]


In [44]:
from data_KD import Sentence
data_wolabel = []
for sent,l in zip(sentences_conll,logits):
    data_wolabel.append(Sentence(sent,logits=l))

data_wlabel = corpus.train

for sent_wl,sent_wol in zip(data_wlabel ,data_wolabel):
    for token_wl,token_wol in zip(sent_wl,sent_wol):
        tag = token_wl.get_tag('ner')
        token_wol.add_tag_label('ner',tag)

import pickle
with open(path+'data/data_conll_LogitsLabel_12.7k.pickle', 'wb') as handle:
    pickle.dump(data_wolabel, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
data_wolabel [13].get_spans('ner')

[<MISC-span (5): "deutscher">, <ORG-span (10,11): "Roten Armee">]

# prepare logits results and get output --- news data

In [3]:
import pickle 
from flair.data import Sentence
from tqdm import tqdm

logits = []

for sent in tqdm(sentences_news):
    sent = Sentence(sent) 
    feature = teacher.forward([sent]).detach() #NOTE: forward function takes batch as input, here seen as batch_size=1, add extra list dim.
    logits.append(feature.squeeze(0)) #NOTE: squeeze the additional 1 dim caused by batch_size 1, but not the additional 1 dim caused by len(sentence) = 1


100%|██████████| 12704/12704 [1:45:28<00:00,  2.01it/s]


In [4]:
from data_KD import Sentence
data_wolabel = []

for sent,l in zip(sentences_news,logits):
    data_wolabel.append(Sentence(sent,logits=l))

for sent in data_wolabel:
    tags,all_tags= teacher._obtain_labels(
        feature=sent.logits.unsqueeze(0),
        batch_sentences=[sent],
        transitions=None, # self.use_crf = False with teacher model
        get_all_tags=False,
    )
    for token,tag in zip(sent.tokens,tags[0]):
        token.add_tag_label('ner',tag)

In [5]:
with open(path+'data/data_news_LogitsLabel_12.7k.pickle', 'wb') as handle:
    pickle.dump(data_wolabel, handle, protocol=pickle.HIGHEST_PROTOCOL)